In [ ]:
import numpy as np
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
check = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer = AutoTokenizer.from_pretrained(check)
# download and cache pre-trained model
model = AutoModelForCausalLM.from_pretrained(check)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Make ChatBot class
class ChatBot():
    # initialize
    def __init__(self):
        # once chat starts, history is saved
        self.chat_history_ids = None
        # input ids are global
        self.bot_input_ids = None
        # check if end the conversation
        self.end_chat = False
        # greet at start
        self.welcome()


    def welcome(self):
        print("Starting Le ChatBot 1.0...")
        # prep user
        time.sleep(2)
        print('Type "bye" or "quit" or "exit" to end chat \n')
        # time for user to read
        time.sleep(3)
        # Greet
        greeting = np.random.choice([
            "Bonjour, I am a ChatBot, call me Bernard.",
            "Hello, let's chat",
            "Nice to meet you, what would you like to chat about?",
            "Hi, I am Bernard. Let's chat."
        ])

        print("ChatBot Bernard >>  " + greeting)

    def user_input(self):
        # get input
        text = input("User    >> ")
        # user can end chat
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            # flag on
            self.end_chat=True
            # ending
            print('ChatBot Bernard>>  Have a lovely day. Good bye.')
            time.sleep(1)
            print('\nQuitting Le ChatBot 1.0...')
        else:
            # continue chat, preprocess input text
            # encode new user input, add the eos_token and return tensor in Pytorch
            self.new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, \
                                                       return_tensors='pt')

    def bot_response(self):
        # append the new user input tokens to the chat history
        # if chat has already begun
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1)
        else:
            # if first entry, initialize bot_input_ids
            self.bot_input_ids = self.new_user_input_ids

        # define the new chat_history_ids based on the preceding chats
        # generate response , limit total chat history = 1000 tokens,
        self.chat_history_ids = model.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer.eos_token_id)

        # last ouput tokens from bot
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        #  if bot doesn't answer
        if response == "":
            response = self.random_response()
        # print bot response
        print('ChatBot Bernard>>  '+ response)

    # if no response from model
    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # iterate over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # answer ques
        if response.strip() == '?':
            reply = np.random.choice(["I don't know",
                                     "I am not sure"])
        # not a question? answer suitably
        else:
            reply = np.random.choice(["Brilliant.",
                                      "Cool. What's up?",
                                      "Alright."
                                     ])
        return reply

In [19]:
# chatbot object
bot = ChatBot()
# begin chat
while True:
    # get user input
    bot.user_input()
    # to end chat or not
    if bot.end_chat:
        break
    # bot response
    bot.bot_response()

Starting Le ChatBot 1.0...
Type "bye" or "quit" or "exit" to end chat 

ChatBot Bernard >>  Nice to meet you, what would you like to chat about?
User    >> Where are you?
ChatBot Bernard>>  I'm in the UK
User    >> Are you in London?
ChatBot Bernard>>  No, I'm in the US
User    >> Are you in New York?
ChatBot Bernard>>  No, I'm in New Jersey
User    >> Do you like New Jersey Nets?
ChatBot Bernard>>  I like New Jersey Nets.
User    >> What about Brooklyn Nets?
ChatBot Bernard>>  I like Brooklyn Nets.
User    >> Do you eat pizza?
ChatBot Bernard>>  I like pizza.
User    >> Can you play music?
ChatBot Bernard>>  I can play music.
User    >> Do you listen to jazz?
ChatBot Bernard>>  I can play jazz.
User    >> What instrument can you play?
ChatBot Bernard>>  I can play guitar.
User    >> Do you speak French?
ChatBot Bernard>>  I can play guitar.
User    >> How have you been?
ChatBot Bernard>>  Alright.
User    >> See you later.
ChatBot Bernard>>  Cool. What's up?
User    >> Bye
ChatBot Ber